## <center>Assignment - Syntactic Analysis</center>
### <center>HMMs and Viterbi algorithm for POS tagging</center>
#### <div style="text-align: left"> Mukundan A P</div>  <div style="text-align: right"> 17th February 2020</div>

## Part-of-Speech Tagging using modified Viterbi

### Data Preparation

In [1]:
# Importing libraries

import nltk
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pprint, time
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
# reading the Treebank tagged sentences

nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# Review the Data

print(nltk_data[:5])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [4]:
# Training & Validation - Ratio 95:5  Splitting

random.seed(1234)
train_set,test_set = train_test_split(nltk_data,train_size=0.95,test_size=0.05,random_state = 101)

In [5]:
# Getting the train tagged words list

train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95547

In [6]:
# Getting the Test tagged words list

test_tagged_words = [tup[0] for sent in test_set for tup in sent]
print(len(test_tagged_words))

5129


In [7]:
# Glimpse of Train Tagged Words

train_tagged_words[:10]

[('Reliance', 'NOUN'),
 ('confirmed', 'VERB'),
 ('the', 'DET'),
 ('filing', 'NOUN'),
 ('but', 'CONJ'),
 ('would', 'VERB'),
 ("n't", 'ADV'),
 ('elaborate', 'VERB'),
 ('.', '.'),
 ('*', 'X')]

In [8]:
# Glimpse of Test Tagged Words

test_tagged_words[:10]

['The', 'company', 'said', '0', 'it', 'is', 'in', 'the', 'process', 'of']

In [9]:
# POS tags for tokens in  Train Set

train_data_pos_tags = [pair[1] for pair in train_tagged_words]
train_data_pos_tags[:10]

['NOUN', 'VERB', 'DET', 'NOUN', 'CONJ', 'VERB', 'ADV', 'VERB', '.', 'X']

In [10]:
# Training Data - Unique Tag Words

tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

12
{'X', 'CONJ', 'ADP', 'VERB', 'DET', 'PRON', '.', 'ADJ', 'NOUN', 'PRT', 'ADV', 'NUM'}


In [11]:
# Unique Words in Vocabulary 

vocabulary = {word for word,tag in train_tagged_words}
print(len(vocabulary))

12100


In [12]:
# Tags and Vocabulary 

print("Length : \nVocabulary: {} \nTags: {}".format(len(vocabulary), len(train_data_pos_tags)))
print("\nAvailable Tags :\n")
print(train_data_pos_tags)

Length : 
Vocabulary: 12100 
Tags: 95547

Available Tags :

['NOUN', 'VERB', 'DET', 'NOUN', 'CONJ', 'VERB', 'ADV', 'VERB', '.', 'X', 'VERB', 'ADJ', 'NOUN', '.', 'ADP', 'ADP', 'DET', 'NOUN', '.', 'DET', 'NOUN', 'NOUN', 'VERB', 'VERB', 'X', 'PRT', 'VERB', 'NOUN', 'X', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'ADP', 'DET', 'NOUN', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'ADV', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ', 'NOUN', '.', 'PRON', 'PRON', 'VERB', 'X', 'VERB', 'ADP', 'X', 'VERB', 'DET', 'NOUN', 'ADP', 'PRON', 'ADJ', 'NOUN', '.', '.', 'NOUN', 'NOUN', 'VERB', 'DET', 'ADJ', '.', 'ADJ', 'NOUN', 'NOUN', '.', 'CONJ', 'NOUN', 'VERB', 'X', 'PRT', 'VERB', 'X', 'PRON', 'VERB', 'VERB', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADV', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'NUM', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'DET', 'NOUN', 'VERB', 'X', 'NOUN', 'NOUN', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADP', '.', 'NUM', 'NUM', 'X', '.', 'ADV', 'ADV', 'ADJ', 'ADP', '

## Part-of-Speech Tagging Algorithm using HMM

HMM - Hidden Markov Model based algorithm is used to tag the words. Given a sequence of words to be tagged, the task is to assign the most probable tag to the word. 

In other words, to every word **w**, assign the tag **t** that maximises the likelihood **P(t/w)**. Since **P(t/w) = P(w/t). P(t) / P(w)**, after ignoring **P(w)**, we have to compute **P(w/t) and P(t)**.


**P(w/t)** is basically the probability that given a tag (say NN), what is the probability of it being w (say 'building'). This can be computed by computing the fraction of all NNs which are equal to w, i.e. 

**P(w/t) = count(w, t) / count(t)**. 


The term **P(t)** is the probability of tag **t**, and in a tagging task, we assume that a tag will depend only on the previous tag. In other words, the probability of a tag being NN will depend only on the previous tag **t(n-1)**. So for e.g. if **t(n-1)** is a **JJ**, then **t(n)** is likely to be an **NN** since adjectives often precede a noun (blue coat, tall building etc.).


Given the penn treebank tagged dataset, we can compute the two terms **P(w/t) and P(t)** and store them in two large matrices. The matrix of **P(w/t)** will be sparse, since each word will not be seen with most tags ever, and those terms will thus be zero. 

### Build the vanilla Viterbi based POS tagger

#### Emission Probabilities for a given word - Computational Function

In [13]:
# Compute Emission probability for a given word for a given tag

def word_given_tag(word,tag,train_bag= train_tagged_words):
    taglist = [pair for pair in train_bag if pair[1] == tag]
    tag_count = len(taglist)    
    w_in_tag = [pair[0] for pair in taglist if pair[0]==word]    
    word_count_given_tag = len(w_in_tag)    
    
    return (word_count_given_tag,tag_count)

#### Function to compute transition probabilties for a given tag and previous tag

In [14]:
# compute transition probabilities of a previous and next tag

def t2_given_t1(t2,t1,train_bag=train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    
    t1_tags = [tag for tag in tags if tag==t1]
    
    count_of_t1 = len(t1_tags)
    
    t2_given_t1 = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    
    count_t2_given_t1 = len(t2_given_t1)
    
    return(count_t2_given_t1,count_of_t1)

In [15]:
# Observing the Noun and Determinants

t2_given_t1('NOUN','DET')

(5284, 8281)

In [16]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [17]:
# Matrix to Data Frame

tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

In [18]:
# Observe the tags in the data frame

tags_df

,X,CONJ,ADP,VERB,DET,PRON,.,ADJ,NOUN,PRT,ADV,NUM
X,0.076384,0.010662,0.142584,0.203851,0.054742,0.055538,0.163590,0.017187,0.062381,0.185232,0.024984,0.002864
CONJ,0.008833,0.000465,0.052534,0.156671,0.121339,0.058113,0.034868,0.118085,0.349140,0.004649,0.055323,0.039981
ADP,0.034427,0.000962,0.016893,0.008340,0.324709,0.070031,0.039025,0.107024,0.320967,0.001390,0.014006,0.062226
VERB,0.217506,0.005577,0.092022,0.169249,0.134392,0.035786,0.034934,0.064988,0.110070,0.030674,0.081952,0.022851
DET,0.045405,0.000483,0.009540,0.039850,0.005676,0.003744,0.017993,0.204323,0.638087,0.000242,0.012438,0.022220
PRON,0.089969,0.005360,0.022971,0.485452,0.009954,0.007657,0.040965,0.073124,0.210949,0.013017,0.034074,0.006508
.,0.026971,0.057538,0.091342,0.089095,0.173335,0.066349,0.093320,0.043963,0.222242,0.002427,0.052324,0.081003
ADJ,0.021091,0.016971,0.078267,0.011699,0.004943,0.000330,0.063931,0.066403,0.699621,0.010710,0.004778,0.021256
NOUN,0.029175,0.042666,0.176514,0.147667,0.012942,0.004607,0.240604,0.012248,0.263564,0.043397,0.017074,0.009542
PRT,0.013509,0.002306,0.020099,0.405272,0.097858,0.017792,0.043822,0.083031,0.247776,0.001647,0.010214,0.056672


#### Viterbi Algorithm

The steps are as follows:

1. Given a sequence of words
2. iterate through the sequence
3. for each word (starting from first word in sequence) calculate the product of emission probabilties and transition probabilties for all possible tags.
4. assign the tag which has maximum probability obtained in step 3 above.
5. move to the next word in sequence to repeat steps 3 and 4 above.

In [19]:
# Vanilla Viterbi Algorithm

def Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Testing Vanilla Viterbi Algorithm on sample test data

In [20]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents

rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents

test_run = [test_set[i] for i in rndom]

# list of tagged words

test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words

test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [21]:
# tagging the test sentences

start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  39.384735107421875
Vanilla Viterbi Algorithm Accuracy:  89.38053097345133


In [22]:
# let's check the incorrectly tagged words

[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('Contra', 'X'), ('Contra', 'NOUN')),
 (('command', 'VERB'), ('command', 'NOUN')),
 (('Honduras', 'X'), ('Honduras', 'NOUN')),
 (('Sandinista', 'X'), ('Sandinista', 'NOUN')),
 (('offensive', 'X'), ('offensive', 'NOUN')),
 (('rebel', 'X'), ('rebel', 'NOUN')),
 (('forces', 'VERB'), ('forces', 'NOUN')),
 (('Bucking', 'X'), ('Bucking', 'VERB')),
 (('drew', 'X'), ('drew', 'VERB')),
 (('Eveready', 'X'), ('Eveready', 'NOUN')),
 (('complaining', 'X'), ('complaining', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]

### Solve the hazels of unknown words

We can see that all of unknown words have been tagged as 'NUM' as 'NUM' is the first tag in tag list and is assigned
if unknown word is encountered (emission probability =0).

### Viterbi Modification-Technique I

* **First solution for unknown words**: assign based on transition probabilities only in case of unknown words as emission probability for unknown word is zero.

In [23]:
# use transition probability of tags when emission probability is zero (in case of unknown words)

def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [24]:
# tagging the test sentences

start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time taken in seconds:  30.958407402038574
Modified Viterbi_1 Accuracy:  94.69026548672566


**Adding Tag occurance probability weights**: 
we will apply weights based on the probability of tag occurance to the transition probabilities of tags and then use the resulting probability for predicting unknown words. 

This scheme will also take into account that some POS tags are more likely to occur as compared to others.

In [25]:
# lets create a list containing tuples of POS tags and POS tag occurance probability, based on training data

tag_prob = []
total_tag = len([tag for word,tag in train_tagged_words])
for t in tags:
    each_tag = [tag for word,tag in train_tagged_words if tag==t]
    tag_prob.append((t,len(each_tag)/total_tag))

tag_prob

[('X', 0.06576867928872701),
 ('CONJ', 0.02251248076862696),
 ('ADP', 0.0978889970381069),
 ('VERB', 0.1351167488251855),
 ('DET', 0.08666938784053921),
 ('PRON', 0.0273373313657153),
 ('.', 0.11641391147812072),
 ('ADJ', 0.06351847781719991),
 ('NOUN', 0.2862674913916711),
 ('PRT', 0.03176447193527793),
 ('ADV', 0.031597015081582885),
 ('NUM', 0.035145007169246546)]

In [26]:
def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
       
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p             
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use weighted transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]                 
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [27]:
# tagging the test sentences

start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time taken in seconds:  29.580455780029297
Modified Viterbi_1 Accuracy:  95.57522123893806


**Thus, we see that we have got a much better accuracy by using weighted transition probabilties.**

In [28]:
# let's check the incorrectly tagged words

[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('Sandinista', 'VERB'), ('Sandinista', 'NOUN')),
 (('Eveready', 'VERB'), ('Eveready', 'NOUN')),
 (('*T*-252', 'VERB'), ('*T*-252', 'X')),
 (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_1 as compared to vanilla Viterbi Algorithm:

Contra:correctly tagged as NOUN
Honduras:correctly tagged as NOUN
complaining: correctly tagged as VERB
Bucking: correctly tagged as VERB

### Viterbi Modification-Technique II

**second solution for unknown words:** 

* backoff to rule based tagger in case of unknown words.
* we further observe that POS tag 'X' can be easily encapsulated in regex rule, so we extract it only based on ruled based tagged.

Let's define a rule based tagger as below:

In [29]:
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [30]:
# Modification in Viterbi Algorithm : Backoff to rule based tagger in case unknown word is encountered.

def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
       
        
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
            
        
        state.append(state_max)
    return list(zip(words, state))

In [31]:
# Test Sentences Tag

start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  33.337754011154175


In [32]:
# Accuracy 

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_2 Accuracy: ',accuracy*100)

Modified Viterbi_2 Accuracy:  97.34513274336283


In [33]:
# Review incorrectly tagged words

[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('drew', 'NOUN'), ('drew', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:

* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB

the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1

* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* `*T*-252`: correctly tagged as 'X'

**further modification in Viterb_2:** We know that the rule based tagger assigns 'NOUN' by default if word does not fall in any rule, to correct this let's assign the tags for any such word based purely on transition probability of tags.

So, first we will modify the rule based tagger to output 'NN' instead of 'NOUN' in case word does not satisfy any rules. We also observe that any capitalized word can still be defaulted as 'NOUN' so will add one more rule for that case.

In [34]:
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'^[A-Z][a-z].*', 'NOUN'),       # NOUN
    (r'.*', 'NN')                     # default
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [35]:
# modified Viterbi

def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1] 
        
      
        # getting state for which probability is maximum
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
            
            # if unknown word does not satisfy any rule, find the tag with maximum transition probability
            if state_max == 'NN':
                pmax = max(p_transition)
                state_max = T[p_transition.index(pmax)]                 
                
        else:
             if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [36]:
# Test Sentences Tagging

start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  30.827258825302124


In [37]:
# Accuracy

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Modified Viterbi Algorithm Accuracy:  98.23008849557522


We observe that much better accuracy is obtained now.

In [38]:
# Observing the Incorrectly tagged words

[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')), (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:
* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB

the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1
* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* `*T*-252`: correctly tagged as 'X'
* drew: correctly tagged as VERB

#### Evaluate vanilla Viterbi and modified Viterbi on entire test data

In [39]:
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
test_run_base = [tup for sent in test_set for tup in sent]

In [ ]:
# Tagging Test Sentences

start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# Accuracy

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

In [ ]:
# Tagging Test Sentences

start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# Accuracy

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

#### Evaluating tagging on sample 'Test_sentences.txt' file

In [ ]:
TS = open('Test_sentences.txt')

In [ ]:
text = TS.read()

In [ ]:
sample_test_sentences = text.splitlines()

In [ ]:
TS.close()

In [ ]:
sample_test_sentences = test_sentences[:-3]

In [ ]:
sample_test_sentences

In [ ]:
# list of untagged words

sample_test_words = [word for sent in sample_test_sentences for word in sent.split()]

In [ ]:
# tagging the test sentences

start = time.time()
sample_tagged_seq = Viterbi(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

In [ ]:
sample_tagged_seq

In [ ]:
We can see that several words have been misclassified by vanilla Viterbi POS tagger, for example:
* Android as NUM
* Google as NUM
* OS as NUM

In [ ]:
# tagging the test sentences

start = time.time()
sample_tagged_seq = Viterbi_2(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

In [ ]:
sample_tagged_seq

In [ ]:
All these cases were correctly POS tagged by Viterbi_2:
* Android as NOUN
* Google as NOUN
* OS as NOUN

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

The accuracy of vanilla Viterbi Algorithm: **91.52%**

The accuracy of modified Viterbi Algorithm: **95.44%**

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

The following cases were incorrectly tagged which got corrected by modified Viterbi Algorithm:

* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB
* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* `*T*-252`: correctly tagged as 'X'
* drew: correctly tagged as VERB